### Importing Libraries

In [64]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

### ETL Pipeline for Client ComZ input feature

In [65]:
#Reading the data from CSV and creating target dataframe with unique user IDs
Data_visit = pd.read_csv(r'C:\Users\steph\Downloads\data (1)\data\VisitorLogsData.csv')
Data_user = pd.read_csv(r'C:\Users\steph\Downloads\data (1)\data\userTable.csv')
print("Data imported")

# Creating the target dataframe with the unique values in the dataset
Data_target = pd.DataFrame(sorted(Data_visit['UserID'].dropna().drop_duplicates()),columns=['UserID'])
print("Target Dataframe Created")

#Picking the dataframe with valid users
print("Indexing valid users")
Data_visit = Data_visit[Data_visit['UserID'].notnull()]
print("Indexing valid users - Done")

# Defining the start date and end date for the dataframe
Current_date = pd.to_datetime('2018-05-28') #System date can be passed in the future
Current_date = datetime.date(Current_date)
Start_date = pd.to_datetime(Current_date)-pd.to_timedelta("21day")
End_date = pd.to_datetime(Current_date)-timedelta(1)
End_date = pd.to_datetime(datetime.combine(datetime.date(End_date),datetime.time(End_date).max))

#Dropping duplicates
Data_visit.drop_duplicates(inplace=True)

# Function to clean the date columns
def date_clean(Date_string):
    if 'nan' in str(Date_string):
        Date_string = np.nan
#         Date_string = Date_string.strftime('%Y-%m-%d %H:%M:%S.%f')
    elif ":" not in str(Date_string):
        Date_string = int(Date_string)
        Date_string = datetime.fromtimestamp(Date_string//1000000000)
        Date_string = Date_string.strftime('%Y-%m-%d %H:%M:%S.%f')
    else:
        Date_string = datetime.strftime(pd.to_datetime(Date_string),'%Y-%m-%d %H:%M:%S.%f')
    return(Date_string)

#Clean Date column
print("Cleaning date column")
Data_visit['VisitDateTime'] = Data_visit['VisitDateTime'].apply(date_clean)
print("Cleaning date column - Done")

# Converting the dataframe to upper case
print("Case cleaning for the dataframe")
Data_visit = Data_visit.fillna('').astype(str).apply(lambda x: x.str.upper())
Data_visit = Data_visit.replace('',np.nan)
print("Case cleaning for the dataframe - Done")


print("Imputing date,Product and activity columns - Executing")
def median(x):
    x = max(x) - ((max(x)-min(x))/2)
    x = pd.to_datetime(x)
    return(x)

def clean_date(Dataframe,index,date_col,product_col,activity_col):
    Dataframe[date_col] = pd.to_datetime(Dataframe[date_col])
    Dataframe = Dataframe.sort_values([index,date_col])
    Mask_1 = Dataframe[product_col].isna()
    Fill_column_1 = Dataframe[product_col].ffill(limit=3)
    Mask_11 = Fill_column_1.eq(Dataframe[product_col].bfill(limit=3))
    Dataframe[product_col] = Dataframe[product_col].mask(Mask_1&Mask_11,Fill_column_1)
    
    Dataframe['Year_Month_Date']=Dataframe.groupby([index,product_col])[date_col].apply(lambda x:x.fillna(median(x)))
    Dataframe[date_col] = Dataframe[date_col].transform(lambda x: x.fillna(Start_date))
    Dataframe['Year_Month_Date']=Dataframe['Year_Month_Date'].transform(lambda x:x.fillna(Dataframe[date_col]))
    
    Dataframe = Dataframe.sort_values([index,'Year_Month_Date'])
    Dataframe['Cumsum'] = (Dataframe[product_col]!=Dataframe[product_col].shift(1)).cumsum()
    
    Mask_2 = Dataframe[product_col].notnull()
    Fill_column_2 = Dataframe.groupby([index,product_col,'Cumsum'])[activity_col].ffill()
    Dataframe[activity_col] = Dataframe[activity_col].mask(Mask_2,Fill_column_2)
    
    Dataframe = Dataframe.sort_values([index,product_col,'Year_Month_Date'])
    Dataframe['Cumsum'] = (Dataframe[product_col]!=Dataframe[product_col].shift(1)).cumsum()
    
    Mask_3 = Dataframe[product_col].notnull()
    Fill_column_3 = Dataframe.groupby([index,product_col,'Cumsum'])[activity_col].ffill()
    Dataframe[activity_col] = Dataframe[activity_col].mask(Mask_3,Fill_column_3)
       
    return(Dataframe)
Data_visit = clean_date(Data_visit,'UserID','VisitDateTime','ProductID','Activity')
print("Imputing date,Product and activity columns - Done")


print("........Dataframe to previous 21 days - Executing..........")
#Slicing the dataframe for previous 21 days
Data_visit = Data_visit[Data_visit['Year_Month_Date'].between(Start_date,End_date)]
print("........Dataframe to previous 21 days - Done..........")


print(".....Executing functions for feature exrtaction........")

#Converting the date column to datetype
Data_visit['Year_Month_Date'] = pd.to_datetime(Data_visit['Year_Month_Date'])

def visits_7_days(Dataframe,index,date_col):
    #getting the dataframe sliced to 7 days from the current date
    Dataframe = Dataframe[Dataframe[date_col] >pd.to_datetime(Current_date)-pd.to_timedelta("7day")]
    #Getting the number of days visited in the last 7 days
    Data = Dataframe.copy()
    Data[date_col] = Data[date_col].apply(lambda x:datetime.date(pd.to_datetime(x)))
    Data = Data.groupby(index).agg({date_col :"nunique"}).\
                rename(columns = {'Year_Month_Date':'No_of_days_Visited_7_Days'}).reset_index()
    return(Data)


def No_Of_Products_Viewed_15_Days(Dataframe,index,date_column,col_target):
    #getting the dataframe sliced to 15 days from the current date
    Dataframe = Dataframe[Dataframe[date_column] >pd.to_datetime(Current_date)-pd.to_timedelta("15day")]
    #Getting the number of days visited in the last 15 days
    Data1 = Dataframe.groupby(index).agg({col_target:"nunique"}).\
                rename(columns = {'ProductID':'No_Of_Products_Viewed_15_Days'}).reset_index()
    return(Data1)

def User_Vintage(Dataframe,date_col,index):
    #converting the string column of the dataframe to dateformat to perform the difference in days
    Dataframe[date_col] = Dataframe[date_col].apply(lambda x:datetime.date(pd.to_datetime(x)))
    Dataframe['User_Vintage'] = Current_date-Dataframe[date_col]
    Dataframe['User_Vintage'] = Dataframe['User_Vintage'].apply(lambda x: x.days)
    return(Dataframe[[index,'User_Vintage']])

def Most_Viewed_product_15_Days(Dataframe,index,filter_col,target_col,date_col):
    #Converting the date column to date format and slicing last 15 days
    Dataframe = Dataframe[Dataframe[date_col] >pd.to_datetime(Current_date)-pd.to_timedelta("15day")]
    
    #Preprocessing the dataframe for the conditions
    Dataframe = Dataframe[Dataframe[filter_col] == 'PAGELOAD']
    
    #Grouping the data and selecting the most viewed product
    temp = Dataframe.groupby([index,target_col]).agg({target_col:'count',date_col:'max'})\
            .rename(columns = {target_col:'Value_count'}).sort_values([index,'Value_count',date_col],ascending = False)\
            .reset_index()
    Data_3 = temp.groupby(index)[index,target_col].head(1).reset_index(drop=True)\
                .rename(columns={target_col:'Most_Viewed_product_15_Days'})
    return(Data_3)

def Most_Active_OS(Dataframe,index,target_col,date_col):
    temp = Dataframe.groupby([index,target_col]).agg({target_col:'count',date_col:'max'})\
            .rename(columns = {target_col:'Value_count'}).sort_values([index,'Value_count',date_col],ascending = False)\
            .reset_index()
    Data_4 = temp.groupby(index)[index,target_col].head(1).reset_index(drop=True)\
                .rename(columns={target_col:'Most_Active_OS'})
    return(Data_4)

def Recently_Viewed_Product(Dataframe,index,filter_col,target_col,date_col):
    #Preprocessing the dataframe for the conditions
    Dataframe = Dataframe[Dataframe[filter_col] == 'PAGELOAD']
    
    #Grouping the data and selecting the most viewed product
    temp = Dataframe.groupby([index,target_col]).agg({date_col:'max'})\
            .rename(columns = {target_col:'Value_count'}).sort_values([index,date_col],ascending = False)\
            .reset_index()
    Data_5 = temp.groupby(index)[index,target_col].head(1).reset_index(drop=True)\
                .rename(columns={target_col:'Recently_Viewed_Product'})
    return(Data_5)

def Pageloads_last_7_days(Dataframe,index,date_col,filter_col):
  
    Dataframe = Dataframe[Dataframe[date_col] >pd.to_datetime(Current_date)-pd.to_timedelta("7day")]
    Dataframe = Dataframe[Dataframe[filter_col] == 'PAGELOAD']
    Data_6 = Dataframe.groupby([index]).agg({index:'count'}).rename(columns={'UserID':'Pageloads_last_7_days'}).reset_index()
    return(Data_6)

def Clicks_last_7_days(Dataframe,index,date_col,filter_col):
   
    Dataframe = Dataframe[Dataframe[date_col] >pd.to_datetime(Current_date)-pd.to_timedelta("7day")]
    Dataframe = Dataframe[Dataframe[filter_col] == 'CLICK']
    Data_7 = Dataframe.groupby([index]).agg({index:'count'}).rename(columns={'UserID':'Clicks_last_7_days'}).reset_index()
    return(Data_7)



print(".........Merging the data........")

#Merging the visitors for last 7 days to the target dataframe
Data_target = Data_target.merge(visits_7_days(Data_visit,'UserID','Year_Month_Date'),how='left',on='UserID')
Data_target['No_of_days_Visited_7_Days'].fillna(0,inplace=True)
Data_target['No_of_days_Visited_7_Days'] = Data_target['No_of_days_Visited_7_Days'].astype(int)

#Merging the products viewed for last 15 days to the target dataframe
Data_target = Data_target.merge(No_Of_Products_Viewed_15_Days(Data_visit,'UserID','Year_Month_Date','ProductID'),how='left',on='UserID')
Data_target['No_Of_Products_Viewed_15_Days'].fillna(0,inplace=True)
Data_target['No_Of_Products_Viewed_15_Days'] = Data_target['No_Of_Products_Viewed_15_Days'].astype(int)

#merging the User_vintage to the target dataframe
Data_target = Data_target.merge(User_Vintage(Data_user,'Signup Date','UserID'),how='left',on='UserID')
Data_target['User_Vintage'].fillna(0,inplace=True)

#merging the Most_Viewed_product_15_Days to the target dataframe
Data_target = Data_target.merge(Most_Viewed_product_15_Days(Data_visit,'UserID','Activity','ProductID','Year_Month_Date')\
                                ,how='left',on='UserID')
Data_target['Most_Viewed_product_15_Days'].fillna("Product101",inplace=True)

#merging the Most_Active_OS to the target dataframe
Data_target = Data_target.merge(Most_Active_OS(Data_visit,'UserID','OS','Year_Month_Date'),how='left',on='UserID')
# Data_target['Most_Active_OS'].fillna('NA',inplace=True)

#merging the Recently_Viewed_Product to the target dataframe
Data_target = Data_target.merge(Recently_Viewed_Product(Data_visit,'UserID','Activity','ProductID','Year_Month_Date')\
                                ,how='left',on='UserID')
Data_target['Recently_Viewed_Product'].fillna("Product101",inplace=True)

#merging the Pageloads_last_7_days to the target dataframe
Data_target = Data_target.merge(Pageloads_last_7_days(Data_visit,'UserID','Year_Month_Date','Activity')\
                                ,how='left',on='UserID')
Data_target['Pageloads_last_7_days'].fillna(0,inplace=True)
Data_target['Pageloads_last_7_days'] = Data_target['Pageloads_last_7_days'].astype(int)

#merging the Clicks_last_7_days to the target dataframe
Data_target = Data_target.merge(Clicks_last_7_days(Data_visit,'UserID','Year_Month_Date','Activity')\
                                ,how='left',on='UserID')
Data_target['Clicks_last_7_days'].fillna(0,inplace=True)
Data_target['Clicks_last_7_days'] = Data_target['Clicks_last_7_days'].astype(int)

print("...............Data merge done...............")


print(".............Exporting to csv - Executing........")
#Printing to CSV
Data_target.to_csv('Jobathon_31.csv',index=False)
print(".............Exporting to csv - Done........")

Data imported
Target Dataframe Created
Indexing valid users
Indexing valid users - Done
Cleaning date column
Cleaning date column - Done
Case cleaning for the dataframe
Case cleaning for the dataframe - Done
Imputing date,Product and activity columns - Executing
Imputing date,Product and activity columns - Done
........Dataframe to previous 21 days - Executing..........
........Dataframe to previous 21 days - Done..........
.....Executing functions for feature exrtaction........
.........Merging the data........


C:\Users\steph\Miniconda3\lib\site-packages\ipykernel_launcher.py:138: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\steph\Miniconda3\lib\site-packages\ipykernel_launcher.py:146: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\steph\Miniconda3\lib\site-packages\ipykernel_launcher.py:158: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


...............Data merge done...............
.............Exporting to csv - Executing........
.............Exporting to csv - Done........
